In [21]:
# Dependencies
import requests
import json
import prettytable
import pandas as pd

In [22]:
# Series IDs are for the categories representative of the metrics used by US Bureau of Labor
# JTU000000480000000JOL: Job openings in the state of Texas in all non-farm industries, with companies of all sizes,
# Values shown in thousands

In [23]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['JTU000000480000000JOL'],"startyear":"2018", "endyear":"2022"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 219,
 'message': [],
 'Results': {'series': [{'seriesID': 'JTU000000480000000JOL',
    'data': [{'year': '2022',
      'period': 'M12',
      'periodName': 'December',
      'latest': 'true',
      'value': '855',
      'footnotes': [{'code': 'P', 'text': 'preliminary'}]},
     {'year': '2022',
      'period': 'M11',
      'periodName': 'November',
      'value': '811',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M10',
      'periodName': 'October',
      'value': '962',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M09',
      'periodName': 'September',
      'value': '1028',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M08',
      'periodName': 'August',
      'value': '971',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M07',
      'periodName': 'July',
      'value': '1115',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M06',
      'peri

In [24]:
# There were 885,000 non-farm job openings in Texas in Dec 2022
json_data["Results"]["series"][0]["data"][0]["value"]

'855'

In [25]:
# Get the year
json_data["Results"]["series"][0]["data"][0]["year"]

'2022'

In [45]:
# Store the year, period, and periodName, and # of job openings in columns of the dataframe
year = []
period = []
periodName = []
value = []
num_series = len(json_data["Results"]["series"])

for i in range(num_series):
    series_data = json_data["Results"]["series"][i]["data"]
    for j in range(len(series_data)):
        year.append(series_data[j]["year"])
        period.append(series_data[j]["period"])
        periodName.append(series_data[j]["periodName"])
        value.append(series_data[j]["value"])



# Create a dataframe from these columns
job_openings = pd.DataFrame({"Year": year,
                            "Period": period,
                            "Month": periodName,
                            "# of job openings (thousands)": value})

# Since the last column contains string values, we convert them to integers
job_openings["# of job openings (thousands)"] = job_openings["# of job openings (thousands)"].astype(int)

# Show # of job openings in the actual thousand values
job_openings["# of job openings"] = job_openings["# of job openings (thousands)"]*1000

# Drop the duplicate column
job_openings = job_openings.drop(columns=["# of job openings (thousands)"])

# Print the dataframe
job_openings

,Year,Period,Month,# of job openings
0,2022,M12,December,855000
1,2022,M11,November,811000
2,2022,M10,October,962000
3,2022,M09,September,1028000
4,2022,M08,August,971000
5,2022,M07,July,1115000
6,2022,M06,June,1038000
7,2022,M05,May,967000
8,2022,M04,April,1104000
9,2022,M03,March,999000
